### Wrangling University of British Columbia Survey Data
In this notebook, I accomplish the following:
* Get an overview of the data.
* Check the Internal ID column for duplicates.
* Clean age values.
* Drop unnecessary columns.
* Combine the four media columns.
* Delete incomplete responses.
* Convert each of the three categories of opinion on each lolly to an integer.
* Standardise the country and state fields.
* 'Unpivot' the data, moving lolly preference columns to rows.


In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
pwd = os.getcwd() # Helps with file management.
survey_df = pd.read_excel(pwd + '\Data - candyhierarchy2017.xlsx')

D:\Personal\Education\Programming\Anaconda1\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Data Overview

In [4]:
survey_df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [5]:
survey_df.describe()

,Internal ID,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
count,2.460000e+03,85.0,1362.0,99.0,67.0
mean,9.028016e+07,1.0,1.0,1.0,1.0
std,7.435532e+03,0.0,0.0,0.0,0.0
min,9.025877e+07,1.0,1.0,1.0,1.0
25%,9.027520e+07,1.0,1.0,1.0,1.0
50%,9.027780e+07,1.0,1.0,1.0,1.0
75%,9.028242e+07,1.0,1.0,1.0,1.0
max,9.031480e+07,1.0,1.0,1.0,1.0


It'll be worth describing the data again once we've converted survey responses to numeric values.

### Checking for Duplicate Respondents
Never trust an assumption about your data until you've verified it, especially when it's this zany!

In [6]:
print(f'Total IDs: {len(survey_df["Internal ID"])}')
print(f'Unique IDs: {survey_df["Internal ID"].nunique()}')

Total IDs: 2460
Unique IDs: 2460


### Clean Age Values
The data contain a few... aberrant responses.

In [7]:
'''
I'll be making a few copies throughout so it's easier to track my progress at different stages and roll back mistakes. 
The tradeoff is storing a bunch of variables, but for a project of this size it's not a real concern.
'''
survey_df_agedrop = survey_df.copy()

In [8]:
print(f'Null values: {survey_df_agedrop["Q3: AGE"].isna().sum()}')
print(f'Non-numeric values: {len([i for i in survey_df_agedrop["Q3: AGE"] if isinstance(i, str) == True])}')
print(f'Total values: {len(survey_df_agedrop["Q3: AGE"])}')

Null values: 84
Non-numeric values: 24
Total values: 2460


We therefore need 108 cells to be 0 whilst preserving the int64 dtype.

In [9]:
survey_df_agedrop['Q3: AGE'] = pd.to_numeric(survey_df_agedrop['Q3: AGE'], errors='coerce')
survey_df_agedrop.replace(np.nan, 0, inplace=True)
survey_df_agedrop.astype({'Q3: AGE': 'int64'}, errors='ignore', copy=False)
survey_df_agedrop['Q3: AGE'].value_counts()

0.0       108
40.0       92
34.0       90
37.0       89
43.0       86
         ... 
88.0        1
312.0       1
70.5        1
99.0        1
1000.0      1
Name: Q3: AGE, Length: 84, dtype: int64

In [10]:
print(f'Null values: {survey_df_agedrop["Q3: AGE"].isna().sum()}')
print(f'Non-numeric values: {len([i for i in survey_df_agedrop["Q3: AGE"] if isinstance(i, str) == True])}')
print(len(survey_df_agedrop['Q3: AGE']))

Null values: 0
Non-numeric values: 0
2460


Now all the rubbish has been swept out, time for a sense-check. It's very unlikely that a 300-year-old is taking this survey. I'm going to assume anyone aged 90 or over is just trying to be funny.

In [11]:
print(f'Number of dubious ages: {len([i for i in survey_df_agedrop["Q3: AGE"] if i >= 90])}')

Number of dubious ages: 8


In [12]:
survey_df_sensecheck = survey_df_agedrop.copy()

In [13]:
survey_df_sensecheck["Q3: AGE"] = [int(i) if i < 90 else 0 for i in survey_df_sensecheck["Q3: AGE"]]
print(f'Number of dubious ages: {len([i for i in survey_df_sensecheck["Q3: AGE"] if i >= 90])}')

Number of dubious ages: 0


I'd also like to exclude a few exceptionally young ages. Halloween lollies are the domain of kids, but I'd say it's highly unlikely any kid under three is capable of coherent opinions. If you were an opinionated two-year-old, don't hesitate to refrain from letting me know.

In [14]:
print(f'Number of far-too-young ages: {len([i for i in survey_df_sensecheck["Q3: AGE"] if i <= 2 and i != 0])}')

Number of far-too-young ages: 1


In [15]:
survey_df_sensecheck["Q3: AGE"] = [int(i) if i > 2 else 0 for i in survey_df_sensecheck["Q3: AGE"]]
print(f'Number of far-too-young ages: {len([i for i in survey_df_sensecheck["Q3: AGE"] if i <= 2 and i != 0])}')

Number of far-too-young ages: 0


To finish up, I'm going to replace all 0s with the median age.

In [16]:
survey_df_sensecheck["Q3: AGE"] = [i if i > 0 else survey_df_sensecheck["Q3: AGE"].median() for i in survey_df_sensecheck["Q3: AGE"]]
survey_df_sensecheck["Q3: AGE"] = survey_df_sensecheck["Q3: AGE"].astype('int64')
survey_df_sensecheck["Q3: AGE"].value_counts()

41    191
40     92
34     90
37     89
43     86
     ... 
8       2
4       1
88      1
74      1
77      1
Name: Q3: AGE, Length: 74, dtype: int64

### Dropping Unneeded Columns
There are several suspect columns in this dataset that are unlikely to be valuable without the aid of quantum computing.

In [17]:
survey_df_sensecheck.columns

Index(['Internal ID', 'Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',
       'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       ...
       'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Q10: DRESS', 'Unnamed: 113',
       'Q11: DAY', 'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]',
       'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]', 'Click Coordinates (x, y)'],
      dtype='object', length=120)

In [18]:
survey_df_sensecheck['Unnamed: 113'].value_counts()

0                                                                          2451
dress (https://survey.ubc.ca/media/assets/user/14372/storage/dress.png)       9
Name: Unnamed: 113, dtype: int64

In [19]:
survey_df_coldrops = survey_df_sensecheck.copy()
survey_df_coldrops.drop(columns=['Q7: JOY OTHER', 'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Unnamed: 113'], inplace=True)
survey_df_coldrops.columns

Index(['Internal ID', 'Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',
       'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       ...
       'Q6 | White Bread', 'Q6 | Whole Wheat anything',
       'Q6 | York Peppermint Patties', 'Q10: DRESS', 'Q11: DAY',
       'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]',
       'Q12: MEDIA [Yahoo]', 'Click Coordinates (x, y)'],
      dtype='object', length=116)

### Combine Media Columns
The media columns are a highly-unrelated portion of the survey where respondents are presented with four mobile homepages and asked to honestly selected which one they'd click on. 

I want a single media column containing a categorical variable for each of the media organisations.

In [20]:
survey_df_media = survey_df_coldrops.copy()

In [21]:
survey_df_media['Q12: MEDIA'] = 0
survey_df_media.head(5)

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)",Q12: MEDIA
0,90258773,0,0,41,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0,"(84, 25)",0
2,90272829,0,Male,49,USA,Virginia,0,0,0,0,...,0,0,0,0,0.0,0.0,0.0,0.0,0,0
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,DESPAIR,DESPAIR,White and gold,Sunday,0.0,1.0,0.0,0.0,"(75, 23)",0
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,DESPAIR,JOY,White and gold,Friday,0.0,1.0,0.0,0.0,"(70, 10)",0


I'm thinking I handle this via the following process:
1. Change media column values to 1 - 4, where 2 would be 'Science', 3 would be 'ESPN' etc.
2. Collapse all values into new MEDIA column.
3. Convert values 1 - 4 into 'Daily Dish' - 'Yahoo' respectively.
4. Drop all four original media columns.

If any of these clowns have selected multiple answers, we'll know soon enough!

In [22]:
def num_changer(data, numadd):
    result = [i + numadd if i > 0 else i for i in data]
    return result

In [23]:
media_data = ['Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]']

for i, media_col in enumerate(media_data):
    survey_df_media[media_col] = num_changer(survey_df_media[media_col], i)

In [24]:
survey_df_media[media_data].loc[1770:1790] # I've taken this oddball slice because it contains responses in each column.

,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1770,0.0,0.0,0.0,0.0
1771,0.0,0.0,0.0,0.0
1772,0.0,0.0,0.0,4.0
1773,0.0,2.0,0.0,0.0
1774,0.0,2.0,0.0,0.0
1775,0.0,0.0,0.0,0.0
1776,0.0,2.0,0.0,0.0
1777,0.0,0.0,0.0,0.0
1778,0.0,2.0,0.0,0.0
1779,0.0,2.0,0.0,0.0


In [25]:
survey_df_media['Q12: MEDIA'] = survey_df_media[media_data].sum(axis=1)
survey_df_media['Q12: MEDIA'].loc[1770:1790]

1770    0.0
1771    0.0
1772    4.0
1773    2.0
1774    2.0
1775    0.0
1776    2.0
1777    0.0
1778    2.0
1779    2.0
1780    0.0
1781    0.0
1782    0.0
1783    2.0
1784    1.0
1785    0.0
1786    2.0
1787    3.0
1788    0.0
1789    0.0
1790    2.0
Name: Q12: MEDIA, dtype: float64

In [26]:
valuedict = {0: 'None',
             1: 'Daily Dish',
             2: 'Science',
             3: 'ESPN',
             4: 'Yahoo'}

for key, value in valuedict.items():
    survey_df_media['Q12: MEDIA'].loc[survey_df_media['Q12: MEDIA'] == key] = value

survey_df_media['Q12: MEDIA'].loc[1770:1790]

D:\Personal\Education\Programming\Anaconda1\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


1770          None
1771          None
1772         Yahoo
1773       Science
1774       Science
1775          None
1776       Science
1777          None
1778       Science
1779       Science
1780          None
1781          None
1782          None
1783       Science
1784    Daily Dish
1785          None
1786       Science
1787          ESPN
1788          None
1789          None
1790       Science
Name: Q12: MEDIA, dtype: object

In [27]:
survey_df_media.drop(columns=media_data, inplace=True)

In [28]:
survey_df_media['Q12: MEDIA'].value_counts()

Science       1362
None           847
ESPN            99
Daily Dish      85
Yahoo           67
Name: Q12: MEDIA, dtype: int64

### Delete Incomplete Responses
Some users opened the survey, but immediately closed the tab. We can figure out who these users are using the gender column. Any response of 0 (which is what we replaced nulls with) must be one of them, as users only had the choice between 'Female', 'Male', 'I'd rather not say' and 'Other'.

In [30]:
rowdrops = survey_df_media[survey_df_media.iloc[:, 4:112:1] == 0]
rowdrops


,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,"Click Coordinates (x, y)",Q12: MEDIA
0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2458,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
